Take the list of Kepler (putative) M dwarfs with at least one confirmed planet and cross-match it with the GAIA DR2 using Megan Bedell's catalog. Then retrieve all 2MASS photometry and compute the stellar parameters. There should be GAIA-derived parameters for at least 206/217 of the Kepler M dwarfs with confirmed planets.

In [2]:
from imports import *
from scipy.interpolate import LinearNDInterpolator as lint
import mwdust, rvs
from get_Kepler_Mdwarf_planets import *
%matplotlib inline

In [12]:
# define some useful functions
def _get_results(samples):
    v = np.percentile(samples, (16,50,84))
    return v[1], v[2]-v[1], v[1]-v[0]
    

def _get_2MASS_Kep(ras_deg, decs_deg, Jmags, Hmags, Kmags,
                   radius_deg=.017, phot_rtol=.02):
    '''Match Kepler stars with GAIA data to the 2MASS point-source catlog to
    retrieve photometric uncertainties.'''
    # get 2MASS data for Kepler stars
    # https://irsa.ipac.caltech.edu/applications/Gator/
    d = np.load('../GAIAMdwarfs/input_data/Keplertargets/fp_2mass.fp_psc12298.npy')
    inds = np.array([0,1,3,5,6,8,9,11])
    ras2M, decs2M, J2M, eJ2M, H2M, eH2M, K2M, eK2M = d[:,inds].T

    # match each star individually
    Nstars = ras_deg.size
    e_Jmags, e_Hmags, e_Kmags = np.zeros(Nstars), np.zeros(Nstars), \
                                np.zeros(Nstars)
    print 'Getting 2MASS photometry...'
    for i in range(Nstars):

        if i % 1e2 == 0:
            print float(i) / Nstars

        # get matching photometry between Kepler-GAIA and 2MASS
        g = (ras2M >= ras_deg[i] - radius_deg) & \
            (ras2M <= ras_deg[i] + radius_deg) & \
            (decs2M >= decs_deg[i] - radius_deg) & \
            (decs2M <= decs_deg[i] + radius_deg) & \
            np.isclose(J2M, Jmags[i], rtol=phot_rtol) & \
            np.isclose(H2M, Hmags[i], rtol=phot_rtol) & \
            np.isclose(K2M, Kmags[i], rtol=phot_rtol)

        if g.sum() > 0:
            g2 = (abs(J2M[g]-Jmags[i]) == np.min(abs(J2M[g]-Jmags[i]))) & \
                 (abs(K2M[g]-Kmags[i]) == np.min(abs(K2M[g]-Kmags[i])))
            e_Jmags[i] = eJ2M[g][g2][0]
            e_Hmags[i] = eH2M[g][g2][0]
            e_Kmags[i] = eK2M[g][g2][0]

        else:
            e_Jmags[i], e_Hmags[i], e_Kmags[i] = np.repeat(np.nan, 3)

    return e_Jmags, e_Hmags, e_Kmags



def _compute_AK_mwdust(ls, bs, dist, edist, eAK_frac=.3, RV=3.1):
    '''Using the EB-V map from 2014MNRAS.443.2907S and the extinction vector
    RK = 0.31 from Schlafly and Finkbeiner 2011 (ApJ 737, 103). 
    Where does RV=3.1 come from? See the MIST BC tables where RV is fixed to 3.1'''
    dustmapK = mwdust.Combined15(filter='2MASS Ks')
    dustmapV = mwdust.Combined15(filter=None)  # returns E(B-V) rather than A_lambda
    dist_kpc, edist_kpc = np.ascontiguousarray(dist)*1e-3, \
                          np.ascontiguousarray(edist)*1e-3
    ls, bs = np.ascontiguousarray(ls), np.ascontiguousarray(bs)
    AK, eAK = np.zeros(ls.size), np.zeros(ls.size)
    AV = np.zeros(ls.size)
    for i in range(ls.size):
        v = dustmapK(ls[i], bs[i],
                     np.array([dist_kpc[i], dist_kpc[i]+edist_kpc[i]]))
        AK[i], eAK[i] = v[0], np.sqrt(abs(np.diff(v))**2 + (eAK_frac*v[0])**2)
        v = dustmapV(ls[i], bs[i],
                     np.array([dist_kpc[i], dist_kpc[i]+edist_kpc[i]]))
        AV[i] = v[0]*RV
    return AK, eAK, AV



def _sample_Rs_from_MK_Mdwarfs(samp_MK):
    '''Use relation from Mann+2015 (table 1) for M dwarfs'''
    assert (np.median(samp_MK) >= 4.6) & (np.median(samp_MK) <= 9.8) 
    a, b, c, Rs_sigma_frac = 1.9515, -.3520, .01680, .0289
    p = np.poly1d((c,b,a))
    samp_MK_tmp = np.copy(samp_MK)
    #samp_MK_tmp[(samp_MK<=4.6) | (samp_MK>=9.8)] = np.nan
    samp_Rs = p(samp_MK_tmp)
    samp_Rs += np.random.normal(0, samp_Rs*Rs_sigma_frac, samp_MK.size)
    return samp_Rs


def _sample_Rs_from_MK_Kdwarfs(samp_MK, samp_Teff, theta):
    '''Use MIST bolometric corrections for stars earlier than M dwarfs.
    theta (most likely values) = Teff, e_Teff, logg, Fe/H, Av'''
    assert len(theta) == 5
    assert np.median(samp_MK) < 4.6
    
    # get bolometric correction from MIST models
    BCK, eBCK = _interpolate_BCK(*theta)
    samp_BCK = np.random.randn(samp_MK.size)*eBCK + BCK
    samp_Mbol = samp_MK - samp_BCK
    
    # compute Rstar
    samp_Lbol = 3.0128e28 * 10**(-.4*samp_Mbol)  # Watts
    sigma = 5.670367e-8
    samp_Rs = rvs.m2Rsun(np.sqrt(samp_Lbol / (4*np.pi*sigma*samp_Teff**4)))
    return samp_Rs


def _sample_Teff_from_colors(samp_GBPmag, samp_GRPmag, samp_Jmag, samp_Hmag,
                             Teff_scatter=49):
    '''Use the relation from Mann+2015 (table 2)'''
    a, b, c, d, e, f, g = 3.172, -2.475, 1.082, -.2231, .01738, .08776, -.04355
    pG = np.poly1d((e,d,c,b,a))
    p2 = np.poly1d((g,f,0))
    samp_Teff = 35e2 * (pG(samp_GBPmag-samp_GRPmag) + p2(samp_Jmag-samp_Hmag)) \
                + np.random.normal(0, Teff_scatter, samp_Jmag.size)
    return samp_Teff


def _sample_Ms_from_MK(samp_MK):
    '''Use relation from Benedict+2016'''
    c0 = np.random.normal(.2311, 4e-4, samp_MK.size)
    c1 = np.random.normal(-.1352, 7e-4, samp_MK.size)
    c2 = np.random.normal(.04, 5e-4, samp_MK.size)
    c3 = np.random.normal(.0038, 2e-4, samp_MK.size)
    c4 = np.random.normal(-.0032, 1e-4, samp_MK.size)
    samp_MK_tmp = np.copy(samp_MK)
    #samp_MK_tmp[(samp_MK<=4.6) | (samp_MK>10)] = np.nan
    #samp_MK_tmp[samp_MK>=10] = np.nan
    dMK = samp_MK_tmp - 7.5
    #samp_Ms = c0 + c1*dMK + c2*dMK**2 + c3*dMK**3 + c4*dMK**4
    # dont care about Ms for this work so don't both distinguishing between M dwarfs and earlier stars
    samp_Ms = np.repeat(samp_MK.size, np.nan)
    return samp_Ms


def _sample_logg(samp_Ms, samp_Rs):
    G = 6.67e-11
    samp_logg = np.log10(G*rvs.Msun2kg(samp_Ms)*1e2 / rvs.Rsun2m(samp_Rs)**2)
    return samp_logg


def _interpolate_BCK(Teff, e_Teff, logg, FeH, Av):
    '''Interpolate the MIST bolometric correction grids to get a star's K-band bolometric correction.'''
    # select a metallicity grid
    FeHgrid = np.array([-4,-3.5,-3,-2.75,-2.5,-2.25,-2,-1.75,-1.5,-1.25,-1,-.75,-.5,-.25,0,.25,.5,.75])
    inds, coeffs = _get_interpolation_coeffs(FeHgrid, FeH)
    if np.any(coeffs == 0):
        FeHs = FeHgrid[inds[coeffs>0]]
        coeffs = np.ones(1)
    else:
        FeHs = FeHgrid[inds]
    
    # interpolate over remaining parameters for all metallicity grids and with sampled Teff
    BCKs = np.zeros((FeHs.size, 2))
    for i in range(FeHs.size):
        
        # get BC grid for a fixed Fe/H
        label = 'p' if FeHs[i] >= 0 else 'm'
        fname = 'UBVRIplus/feh%s%.3d.UBVRIplus'%(label, abs(FeHs[i])*1e2)
        Teffgrid, logggrid, Avgrid, BCKgrid  = np.loadtxt(fname, skiprows=5, usecols=(0,1,3,12)).T
        
        # compute a separate BCK for Teff across its 1 sigma uncertainty to estimate the uncertainty in BCK
        # because it is dominated by uncertainies in Teff which have the strongest effect on the stellar SED
        BCKs_Teff, j = np.zeros(0), 0
        for t in Teff+np.arange(2)*e_Teff:
            # interpolate to get BCK
            lint_BCK = lint(np.array([Teffgrid,logggrid,Avgrid]).T, BCKgrid)
            BCKs[i,j] = np.append(BCKs_Teff, float(lint_BCK(t, logg, Av)) * coeffs[i])
            print BCKs[i,j]
            j += 1
            
    # sum contributions from each Fe/H
    BCKs = np.sum(BCKs, axis=0)
    assert BCKs.size == 2
    BCK, e_BCK = BCKs[0], abs(float(np.diff(BCKs)))
    return BCK, e_BCK

                     
def _get_interpolation_coeffs(arr, val):
    if val < arr.min():
        return np.argsort(arr)[:1], np.ones(1)
    elif val > arr.max():
        return np.argsort(arr)[-1:], np.ones(1)
    else:
        edgeinds = np.argsort(abs(arr-val))[:2]
        edgevals = arr[edgeinds]
        diff = abs(float(np.diff(edgevals)))
        c1 = abs(val-edgevals[0]) / diff
        coeffs = np.array([1-c1, c1]).reshape(2)
        return edgeinds, coeffs

In [13]:
# get Kepler IDs of potential stars of interest confirmed planets
KepMdwarffilein = 'Keplertargets/NASAarchive_confirmed_Keplerlowmassstars.csv'
kepidsM = np.loadtxt(KepMdwarffilein, delimiter=',', skiprows=66, usecols=(1))
N = kepidsM.size
print N

386


In [14]:
# get megan bedell catalog
hdu = fits.open('../GAIAMdwarfs/input_data/Keplertargets/kepler_dr2_4arcsec.fits')[1].data

In [72]:
# setup arrays
GAIAdata = np.zeros(N, dtype=bool)
badGAIA, bad2MASS, baddistpost = np.zeros(N, dtype=bool), np.zeros(N, dtype=bool), np.zeros(N, dtype=bool)
ras, decs = np.zeros(N), np.zeros(N)
GBPmag, e_GBPmag = np.zeros(N), np.zeros(N)
GRPmag, e_GRPmag = np.zeros(N), np.zeros(N)
Kepmag = np.zeros(N)
Jmag, e_Jmag = np.zeros(N), np.zeros(N)
Hmag, e_Hmag = np.zeros(N), np.zeros(N)
Kmag, e_Kmag = np.zeros(N), np.zeros(N)
parallax_mas, e_parallax = np.zeros(N), np.zeros(N)
dist_pc, ehi_dist, elo_dist = np.zeros(N), np.zeros(N), np.zeros(N)
mu, ehi_mu, elo_mu = np.zeros(N), np.zeros(N), np.zeros(N)
AK, e_AK = np.zeros(N), np.zeros(N)
MK, ehi_MK, elo_MK = np.zeros(N), np.zeros(N), np.zeros(N)
isMdwarf = np.zeros(N, dtype=bool)
Rs_RSun, ehi_Rs, elo_Rs = np.zeros(N), np.zeros(N), np.zeros(N)
Teff_K, ehi_Teff, elo_Teff = np.zeros(N), np.zeros(N), np.zeros(N)
Ms_MSun, ehi_Ms, elo_Ms = np.zeros(N), np.zeros(N), np.zeros(N)
logg_dex, ehi_logg, elo_logg = np.zeros(N), np.zeros(N), np.zeros(N)

In [70]:
# retrieve and compute stellar parameters
for i in range(N):

    # is this star in the GAIA-Kep catalog?
    print i, kepidsM[i]
    g = hdu['kepid'] == kepidsM[i]
    GAIAdata[i] = g.sum() > 0

    if g.sum() == 0:
        continue
    
    elif g.sum() > 1:
        ra = hdu['ra'][g]
        dec = hdu['dec'][g]
        print kepidsM[i], ra, dec, hdu['parallax'][g]
        #index = int(raw_input('Which index is correct? '))
        index = int(np.argsort(hdu['parallax'][g])[-1])   # TEMP
        
    else:
        index = 0
        
    # save some stellar parameters
    ras[i] = hdu['ra'][g][index]
    decs[i] = hdu['dec'][g][index]
    l, b = hdu['l'][g][index], hdu['b'][g][index]
    GBPmag[i] = hdu['phot_bp_mean_mag'][g][index]
    FBP = hdu['phot_bp_mean_flux'][g][index]
    eFBP = hdu['phot_bp_mean_flux_error'][g][index]
    e_GBPmag[i] = -2.5*np.log10(FBP / (FBP+eFBP))
    GRPmag[i] = hdu['phot_rp_mean_mag'][g][index]
    FRP = hdu['phot_rp_mean_flux'][g][index]
    eFRP = hdu['phot_rp_mean_flux_error'][g][index]
    e_GRPmag[i] = -2.5*np.log10(FRP / (FRP+eFRP))
    Kepmag[i] = hdu['kepmag'][g][index]
    Jmag[i] = hdu['jmag'][g][index]
    Hmag[i] = hdu['hmag'][g][index]
    Kmag[i] = hdu['kmag'][g][index]
    parallax_mas[i] = hdu['parallax'][g][index] + .029
    e_parallax[i] = hdu['parallax_error'][g][index]

    params = l,b,GBPmag[i],e_GBPmag[i],GRPmag[i],e_GRPmag[i],parallax_mas[i],e_parallax[i]
    if np.any(np.isnan(list(params))):
        print kepidsM[i], params
        badGAIA[i] = True
    
    Nsamp = 1000                 
    samp_GBP = np.random.randn(Nsamp)*e_GBPmag[i] + GBPmag[i]
    samp_GRP = np.random.randn(Nsamp)*e_GRPmag[i] + GRPmag[i]

    # get 2MASS photometric uncertainies
    p = _get_2MASS_Kep(ras[i:i+1], decs[i:i+1], Jmag[i:i+1], Hmag[i:i+1], Kmag[i:i+1])
    if np.any(np.isnan(p)):
        e_Hmag[i], e_Kmag[i] = np.repeat(e_Jmag[i],2)    
    else:
        e_Jmag[i], e_Hmag[i], e_Kmag[i] = p
    samp_J = np.random.randn(Nsamp)*e_Jmag[i] + Jmag[i]
    samp_H = np.random.randn(Nsamp)*e_Hmag[i] + Hmag[i]
    samp_K = np.random.randn(Nsamp)*e_Kmag[i] + Kmag[i]

    params = Jmag[i],e_Jmag[i],Hmag[i],e_Hmag[i],Kmag[i],e_Kmag[i]
    if np.any(np.isnan(list(params))):
        print kepidsM, params
        bad2MASS[i] = True
    
    # get distance posteriors from Bailor-Jones
    try:
        fname='../GAIAMdwarfs/Gaia-DR2-distances_custom/DistancePosteriors/KepID_%i.csv'%(kepidsM[i])
        x_dist, pdf_dist = np.loadtxt(fname, delimiter=',', skiprows=1,
                                      usecols=(1,2)).T
        samp_dist = np.random.choice(x_dist, Nsamp, p=pdf_dist/pdf_dist.sum())
    except IOError:
        # print statement to be run in the script below: save_posteriors(IDnums, pars, e_pars, ls, bs, Kep=True)
        #print 'IDnums,pars,e_pars,ls,bs=np.array([%i]),np.array([%.8e]),np.array([%.8e]),np.array([%.8e]),np.array([%.8e])'%(kepids[i], parallax_mas[i], e_parallax[i], l, b)
        #raise ValueError('Need to compute the distance posterior for KepID_%i (see get_gaia_2MASS.save_posteriors())'%kepids[i])                    
        f = open('tocomputedistance.csv', 'a')
        f.write('%i,%.8e,%.8e,%.8e,%.8e\n'%(kepidsM[i], parallax_mas[i], e_parallax[i], l, b))
        f.close()
        samp_dist = np.repeat(np.nan, Nsamp)
        baddistpost[i] = True    
            
    # compute stellar parameters
    dist_pc[i], ehi_dist[i], elo_dist[i] = _get_results(samp_dist.reshape(Nsamp,1))
    samp_mu = 5*np.log10(samp_dist) - 5
    mu[i], ehi_mu[i], elo_mu[i] = _get_results(samp_mu.reshape(Nsamp,1))
    AK[i], e_AK[i], Av = _compute_AK_mwdust(l, b, dist_pc[i], ehi_dist[i])
    samp_AK = np.random.randn(Nsamp)*e_AK[i] + AK[i]
    samp_MK = samp_K - samp_mu - samp_AK
    MK[i], ehi_MK[i], elo_MK[i] = _get_results(samp_MK.reshape(Nsamp,1))
    
    if (MK[i] >= 4.6) and (MK[i] <= 9.8):
        isMdwarf[i] = True
        samp_Rs = _sample_Rs_from_MK_Mdwarfs(samp_MK)
        Rs_RSun[i], ehi_Rs[i], elo_Rs[i] = _get_results(samp_Rs.reshape(Nsamp,1))
        samp_Teff = _sample_Teff_from_colors(samp_GBP, samp_GRP, samp_J, samp_H)
        Teff_K[i], ehi_Teff[i], elo_Teff[i] = _get_results(samp_Teff.reshape(Nsamp,1))
        #samp_Ms = _sample_Ms_from_MK(samp_MK)
        #Ms_MSun[i], ehi_Ms[i], elo_Ms[i] = _get_results(samp_Ms.reshape(Nsamp,1))
        #samp_logg = _sample_logg(samp_Ms, samp_Rs)
        #logg_dex[i], ehi_logg[i], elo_logg[i] = _get_results(samp_logg.reshape(Nsamp,1))
    
    elif np.isfinite(np.median(samp_MK)):
        isMdwarf[i] = False
        samp_Teff = _sample_Teff_from_colors(samp_GBP, samp_GRP, samp_J, samp_H)
        Teff_K[i], ehi_Teff[i], elo_Teff[i] = _get_results(samp_Teff.reshape(Nsamp,1))
        theta = Teff_K[i], ehi_Teff[i], hdu['logg'][g][index], hdu['feh'][g][index], Av
        samp_Rs = _sample_Rs_from_MK_Kdwarfs(samp_MK, samp_Teff, theta)
        Rs_RSun[i], ehi_Rs[i], elo_Rs[i] = _get_results(samp_Rs.reshape(Nsamp,1))
        #samp_Ms = _sample_Ms_from_MK(samp_MK)
        #Ms_MSun[i], ehi_Ms[i], elo_Ms[i] = _get_results(samp_Ms.reshape(Nsamp,1))
        #samp_logg = _sample_logg(samp_Ms, samp_Rs)
        #logg_dex[i], ehi_logg[i], elo_logg[i] = _get_results(samp_logg.reshape(Nsamp,1))
        
    # save stellar posteriors
    samp_Ms, samp_logg = np.zeros(samp_Rs.size), np.zeros(samp_Rs.size)
    allpost = np.array([samp_GBP,samp_GRP,samp_J,samp_H,samp_K,samp_dist,
                        samp_mu,samp_AK,samp_MK,samp_Rs,samp_Teff,samp_Ms,samp_logg])
    distoutname = '../GAIAMdwarfs/Gaia-DR2-distances_custom/DistancePosteriors/KepID_allpost_%i'%kepidsM[i]
    np.savetxt(distoutname, allpost.T, fmt='%.8e', delimiter=',')

0 1873513.0
Getting 2MASS photometry...
0.0
1 2165002.0
2165002.0 [292.47077636 292.46980222] [37.56720381 37.56779409] [0.28979174 1.29686027]
Getting 2MASS photometry...
0.0
1.2273188525741257
1.20677990565629
0.4794849505371697
0.47152829677770436
2 2165002.0
2165002.0 [292.47077636 292.46980222] [37.56720381 37.56779409] [0.28979174 1.29686027]
Getting 2MASS photometry...
0.0
1.227089036186475
1.2063224082007833
0.4793959472699605
0.47135100305330385
3 2556650.0
2556650.0 [286.5942529  286.59329681] [37.89128358 37.89180683] [ 3.42132918 -0.28369002]
Getting 2MASS photometry...
0.0
4 2581554.0
Getting 2MASS photometry...
0.0
1.0153538543922822
0.9976209878442037
0.8011127867151526
0.7872374665847357
5 2715135.0
Getting 2MASS photometry...
0.0
6 2975770.0
Getting 2MASS photometry...
0.0
1.0401303731461142
1.0232739016769659
0.9632979267800925
0.9477898077063299
7 2987027.0
Getting 2MASS photometry...
0.0
1.827697091863786
1.7996481388393586
8 3234598.0
9 3234598.0
10 3239945.0
32399

KeyboardInterrupt: 

-3.5942042071902183

In [17]:
# save stellar data
hdr = 'KepID,ra_deg,dec_deg,GBPmag,e_GBPmag,GRPmag,e_GRPmag,Kepmag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,parallax_mas,e_parallax,dist_pc,ehi_dist,elo_dist,mu,ehi_mu,elo_mu,AK,e_AK,MK,ehi_MK,elo_MK,Rs_RSun,ehi_Rs,elo_Rs,Teff_K,ehi_Teff,elo_Teff,Ms_MSun,ehi_Ms,elo_Ms,logg_dex,ehi_logg,elo_logg'
outarr = np.array([kepidsM,ras,decs,GBPmag,e_GBPmag,GRPmag,e_GRPmag,Kepmag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,parallax_mas,e_parallax,dist_pc,ehi_dist,elo_dist,mu,ehi_mu,elo_mu,AK,e_AK,MK,ehi_MK,elo_MK,Rs_RSun,ehi_Rs,elo_Rs,Teff_K,ehi_Teff,elo_Teff,Ms_MSun,ehi_Ms,elo_Ms,logg_dex,ehi_logg,elo_logg])
fname = '../GAIAMdwarfs/input_data/Keplertargets/KepMdwarfsv11_archiveplanetsv2.csv'
np.savetxt(fname, outarr.T, delimiter=',', fmt='%.8e', header=hdr)

In [23]:
# check some arrays interactively
g = GAIAdata
b = GAIAdata==False
kepidsM

array([ 1873513.,  2165002.,  2165002.,  2556650.,  2581554.,  2715135.,
        2975770.,  2987027.,  3234598.,  3234598.,  3239945.,  3239945.,
        3239945.,  3245969.,  3426367.,  3444588.,  3546060.,  3554031.,
        3554031.,  3642335.,  3728432.,  3733628.,  3749365.,  3859079.,
        3966801.,  4049131.,  4056616.,  4139816.,  4139816.,  4139816.,
        4139816.,  4142847.,  4180280.,  4249725.,  4249725.,  4263293.,
        4263293.,  4263293.,  4633570.,  4633570.,  4725681.,  4725681.,
        4736569.,  4736644.,  4813563.,  4832837.,  4852528.,  4852528.,
        4852528.,  4852528.,  4852528.,  4913852.,  4917596.,  5080636.,
        5084942.,  5164255.,  5175986.,  5184911.,  5185897.,  5185897.,
        5185897.,  5209845.,  5252423.,  5252423.,  5269467.,  5340644.,
        5364071.,  5364071.,  5364071.,  5364071.,  5371776.,  5371776.,
        5371776.,  5371776.,  5438099.,  5438099.,  5438099.,  5438099.,
        5456651.,  5456651.,  5526527.,  5531576., 